In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from data_gen import get_agent_distribution_nels, generate_losses
from utils import compute_continuity_noise

In [2]:
d=10
prev_beta = np.ones(d)/np.sqrt(d)
prev_s= 0.1

In [3]:
agent_dist, indices = get_agent_distribution_nels(sigma=0.1, prev_beta=prev_beta, prev_s=0.1, n=1000000, subsample=50, seed=0)

In [ ]:
agent_dist.get_gammas()

In [ ]:
compute_continuity_noise(agent_dist)

In [ ]:
def loss_beta(agent_dist, beta, sigma, q):
    s_beta = agent_dist.quantile_fixed_point_true_distribution(beta, sigma, q)
    br_noisy_scores = agent_dist.best_response_noisy_score_distribution(self, beta, s_beta, sigma)
    rejected = br_noisy_scores < np.quantile(br_noisy_scores, q)
    losses, stu_id = generate_losses()
    losses[rejected] = 0.
    return np.sum(losses)

In [ ]:
loss_beta(agent_dist, prev_beta, 0.1, 0.7)